## 1. Setting up Deployment/Test Environment


### 1.1. Setup global variables to be used in the further steps

Please change the resourceGroupName variable to something to a globally unique value. Keep the length as short as ~5 character length.

We recommend that you keep the resource group name less than 5 characters and all in small letters, because: we assign auto generated names for all other resources (i.e. storage account, key vault etc.) to have their name prefixed with resource group name and suffixed with resource type. Some resources have name length limitation and naming convention (like all should be lower case and no specific symbols...). USing invalid characters etc. may cause a failure. So explore the outputs of each cell after running them to see if there is error or successful run.

In [1]:
# Mandatory settings section
# ---------------------------

# Define the ID of your Azure subscription to be used
# You may learn your subscription ID at https://account.azure.com/subscriptions
azureSubsctiptionId = "35c2594a-23da-4fce-b59c-f6fb9513eeeb"

# Define Datacenter location of the azure services to be created.
# To get the list of possible resource locations use the following commands on an Azure CLI enabled terminal window:
#          az account list-locations
resourceLocation = "westus2"  # set any other location that is close to you i.e. "westus2", "southcentralus"

# Name of the resource group that will contain all Azure services to be created
# Resource group name will be appended as a prefix to other resource names, so please keep it to be globally unique and less than 6 characters
resourceGroupName = "mhlva01" # i.e. "mk001"




# Optional settings section
# ---------------------------

# Azure Container Registry Service to host your docker container images
containerRegServiceName = resourceGroupName+'contreg'

# Local Container Registry Service name
localContainerRegServiceName = "mhregistry:55000"

# Azure IoT Hub Service
iotHubServiceName = resourceGroupName+'iothub'

# Name of the container image for ml solution
containerImageName = resourceGroupName+'aimodule'

# Define a folder name that you want to host all your ml solution files
mlSolutionPath = "../src/alpr/lva_ai_solution"

# Define a unique IoT Edge device ID
iotDeviceId = resourceGroupName+'iotdevidlpr'

storageConnStr = "DefaultEndpointsProtocol=https;AccountName=mhlva01store;AccountKey=rlGmLje1O8H3TS8vH4+r0eCEEgYIFf4fzl0LuRpE5ueSag9uMR+vZ3S85S2vcysLWMscTLvt4IlL+dkE6DT1CQ==;EndpointSuffix=core.windows.net"

### 1.2. Check Azure CLI version
Azure CLI is required to run this sample. Check if Azure CLI installed.  
If Azure CLI is not installed, please follow the installation instructions shown here:  
https://docs.microsoft.com/en-us/cli/azure/install-azure-cli-apt?view=azure-cli-latest

In [2]:
!az --version

azure-cli                          2.4.0

command-modules-nspkg              2.0.3
core                               2.4.0
nspkg                              3.0.4
telemetry                          1.0.4

Python location '/home/athena/lib/azure-cli/bin/python'
Extensions directory '/home/athena/.azure/cliextensions'

Python (Linux) 3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]

Legal docs and information: aka.ms/AzureCliLegal


Your CLI is up-to-date.

Please let us know how we are doing: https://aka.ms/clihats
and let us know if you're interested in trying out our newest features: https://aka.ms/CLIUXstudy


### 1.3. Login to Azure
<span style="color:red; font-weight: bold; font-size:1.5em;"> [!Important] </span>  
Run the following command and <span style="color:red; font-weight: bold; font-size:1.1em;"> follow the instructions </span> in the corresponding output cell. You should enter the authentication code to let this notebook kernel be able to access Azure resources. You can run "az logout" command or stop the kernel to logout.

In [3]:
%%bash --out output
az account show &> /dev/null
if [ "$?" != "0" ]; then echo 'login'; fi

In [4]:
if (output != ''):
    print('You must sign in to Azure. Please follow the instructions below.')
    !az login -o None
    output = ''
else:
    print('There is an active Azure login session exist.')

You must sign in to Azure. Please follow the instructions below.
You have logged in. Now let us find all the subscriptions to which you have access...
Failed to authenticate '{'additional_properties': {}, 'id': '/tenants/ec64c433-1ee8-4dbd-837a-39f56c7cdcea', 'tenant_id': 'ec64c433-1ee8-4dbd-837a-39f56c7cdcea', 'tenant_category': <TenantCategory.home: 'Home'>, 'country': None, 'country_code': 'CH', 'display_name': None, 'domains': ['virtualworldmail.buhlergroup.com', 'virtualworld.buhlergroup.com', 'buhlergroup.com', 'ptbuhler.com', 'digitalsense.buhlergroup.com', 'app.buhlergroup.com', 'sft-uzwil.ch', 'buhlergroup.onmicrosoft.com', 'b-point.buhlergroup.com', 'sp01.buhlergroup.com', 'buhlergroup.mail.onmicrosoft.com', 'buhler.id', 'bepex.hosokawa.com', 'buhler-barth.com', 'b-point.buhlertest.ch', 'st01.buhlertest.ch', 'webmaster.buhlergroup.com', 'buhlergroup.io', 'buhlerinsights.buhlergroup.com', 'healthbalancegroup.ch', 'buhlertest.ch', 'bindler.com', 'sanmak.com.br', 'sortex.com', '

### 1.4. Set active Azure Subscription
You may have more than one subscription and we will set the one you specified in **azureSubsctiptionId** as default one to be used.

In [5]:
!az account set --subscription $azureSubsctiptionId

### 1.5. Install dotenv python library to share global variables among code files
dot-env library reads the key-value pair from .env file and adds them to environment variable. It is great for managing app settings during development and in production using 12-factor principles.  

<span style="color:red; font-weight: bold; font-size:1.5em;"> [!Important] </span>  
Depending on your installation you may need to switch between pip and pip3. See cell output below for possible error and make appropriate switch.

In [6]:
%%bash
if python3 -c 'import pkgutil; exit(not pkgutil.find_loader("dotenv"))';
then
	echo "dotenv package found. Skipping re-installation...";
else
	echo "dotenv package not found. Installing...";
    pip3 install python-dotenv;
fi

dotenv package not found. Installing...


<span style="color:red; font-weight: bold; font-size:1.5em;"> [!Important] </span>  
You may have more than one Python3 installation (i.e. Python 3.6, Python 3.8 etc.), so pip3 command may install libraries under specific version which may cause the below cells execution to fail. Be sure that your Jupyter notebook running in the right Pyhon kernel version.  

<img src="../doc_imgs/img_03_025.jpg" width=300/>  

### 1.6. Save global variables in .env file
Check if .env file already exist. If not, create a new one.  
.env file is the place we will store our variables to share with other code books of this sample.  
if you want more details about dotenv python package please refer: https://pypi.org/project/python-dotenv/

In [2]:
from pathlib import Path
from dotenv import set_key, get_key, find_dotenv

envPath = find_dotenv()
if envPath=="":
    Path(".env").touch()
    envPath = find_dotenv()
    
print(".env file path: {0}".format(envPath))

.env file path: /home/athena/Documents/bin/github/curbside/notebooks/.env


In [3]:
set_key(envPath, "azureSubsctiptionId", azureSubsctiptionId)
set_key(envPath, "resourceLocation", resourceLocation)
set_key(envPath, "resourceGroupName", resourceGroupName)
set_key(envPath, "containerRegServiceName", containerRegServiceName)
set_key(envPath, "iotHubServiceName", iotHubServiceName)
set_key(envPath, "mlSolutionPath", mlSolutionPath)
set_key(envPath, "containerImageName", containerImageName)
set_key(envPath, "iotDeviceId", iotDeviceId)
set_key(envPath, "localContainerRegServiceName", localContainerRegServiceName)
set_key(envPath, "storageConnStr", storageConnStr)

(True,
 'storageConnStr',
 'DefaultEndpointsProtocol=https;AccountName=mhlva01store;AccountKey=rlGmLje1O8H3TS8vH4+r0eCEEgYIFf4fzl0LuRpE5ueSag9uMR+vZ3S85S2vcysLWMscTLvt4IlL+dkE6DT1CQ==;EndpointSuffix=core.windows.net')